# Linear PTO

### Options

In [1]:
same_electrical_mechanical = False
use_constraint = False
scale_x_wec = 1.0
scale_x_opt = 1.0
scale_obj = 1.0

# same_electrical_mechanical = True
# use_constraint = False
# scale_x_wec = 1.0
# scale_x_opt = 100.0
# scale_obj = 1.0

# same_electrical_mechanical = True
# use_constraint = True
# scale_x_wec = 1.0
# scale_x_opt = 0.01
# scale_obj = 1.0

### Run

In [2]:
import os
import logging

import matplotlib.pyplot as plt
import capytaine as cpy
from autograd.builtins import isinstance, tuple, list, dict
import autograd.numpy as np

import wecopttool as wot


# logging 
logging.basicConfig(level=logging.INFO)

# create save directory
results_dir = 'results_linear_pto'
if not os.path.exists(results_dir):
  os.makedirs(results_dir)

## WEC object
# mesh
wb = wot.geom.WaveBot()  # use standard dimensions
mesh_size_factor = 0.5  # 1.0 for default, smaller to refine mesh
mesh = wb.mesh(mesh_size_factor)

# floating body
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="HEAVE")

# hydrostatics
hs_data = wot.hydrostatics.hydrostatics(fb)
mass_33 = wot.hydrostatics.mass_matrix_constant_density(hs_data)[2, 2]
mass = np.atleast_2d(mass_33)
stiffness_33 = wot.hydrostatics.stiffness_matrix(hs_data)[2, 2]
stiffness = np.atleast_2d(stiffness_33)

# frequencies
f0 = 0.05
nfreq = 50

# PTO kinematics
kinematics = np.eye(fb.nb_dofs)

# PTO impedance
if same_electrical_mechanical:
    pto_impedance = np.array([[0.0, 1.0], [1.0, 0.0]])
else:
    gear_ratio = 12.4666
    torque_constant = 6.1745
    electrical_constant = 4.116
    winding_resistance = 0.5
    pto_impedance = np.array([[0.0, torque_constant*gear_ratio],
                        [electrical_constant*gear_ratio, winding_resistance]])


# PTO
pto = wot.pto.PseudoSpectralLinearPTO(nfreq, kinematics, pto_impedance)

# constraints
if use_constraint:
    f_max = 2000.0
    nsubsteps = 4


    def const_f_pto(wec, x_wec, x_opt):
        f = pto.force_on_wec(wec, x_wec, x_opt, nsubsteps)
        return f_max - np.abs(f.flatten())


    ineq_cons = {'type': 'ineq',
                'fun': const_f_pto,
                }
    constraints = [ineq_cons]
else:
    constraints = []

# additional dynamics forces function
f_add = pto.force_on_wec

# WEC object
wec = wot.WEC(fb, mass, stiffness, f0, nfreq,
              f_add=f_add, constraints=constraints)

# BEM
fname = os.path.join(results_dir, 'bem.nc')
if os.path.exists(fname):
    wec.read_bem(fname)
else:
    wec.run_bem()
    wec.write_bem(fname)

# waves
wfreq = 0.3
amplitude = 0.0625
phase = -40
waves = wot.waves.regular_wave(f0, nfreq, wfreq, amplitude, phase)

# objective function
obj_fun = pto.electric_average_power
nstate_opt = pto.nstate

# solve
options = {'maxiter': 1000, 'ftol': 1e-8}

wec_tdom, wec_fdom, x_wec, x_opt, obj, res = wec.solve(
    waves, obj_fun, nstate_opt, optim_options=options,
    scale_x_wec=scale_x_wec, scale_x_opt=scale_x_opt, scale_obj=scale_obj)

# post-process
pto_tdom, pto_fdom = pto.post_process(wec, x_wec, x_opt)


INFO:capytaine.bodies.bodies:Stored 1054 triangle faces as quadrilaterals
INFO:capytaine.bodies.bodies:New floating body: WaveBot.


IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

### CC (Mechanical) Solution
Optimal solution for maximum mechanical power, no constraints.

In [ ]:
# CC solution
idof = 0
Fe = wec_fdom['excitation_force'][1:, idof]
Zi = wec.hydro.Zi[:, idof, idof]

cc_vel_fd = Fe / (2*Zi.real)
cc_force_fd = -1.0 * Zi.conj() * cc_vel_fd

cc_vel_td = wot.post_process_continuous_time(cc_vel_fd)
cc_force_td = wot.post_process_continuous_time(cc_force_fd)

### CC (Electrical) Solution
Optimal solution for maximum mechanical power, no constraints.

In [ ]:
Z_11, Z_12 = pto_impedance[0, :]
Z_21, Z_22 = pto_impedance[1, :]
# V_th = Z_21 / (Z_11 + Zi) * Fe
# Z_th = Z_22 - (Z_12*Z_21) / (Z_11 + Zi)
V_th = Z_21 / (Z_11 - Zi) * Fe
Z_th = Z_22 - (Z_12*Z_21) / (Z_11 - Zi)

cc_current_fd = V_th / (2*Z_th.real)
cc_voltage_fd = -1.0 * Z_th.conj() * cc_current_fd 

cc_current_td = wot.post_process_continuous_time(cc_current_fd)
cc_voltage_td = wot.post_process_continuous_time(cc_voltage_fd)


### Results

In [ ]:
# print
print(f"Average mechanical power: {pto.average_power(wec, x_wec, x_opt)} W")
print(f"Average electrical power: {pto.electric_average_power(wec, x_wec, x_opt)} W\n")

In [ ]:
nsubsteps = 10

t = wec.make_time_vec(nsubsteps)

# plot mechanical power
plt.figure()
plt.plot(t, cc_vel_td(t), '-', label='CC_m')
plt.plot(t, pto.velocity(wec, x_wec, x_opt, nsubsteps), '--', label='PS')
plt.xlabel('time [s]')
plt.ylabel('velocity [m/s]')
plt.legend()

plt.figure()
plt.plot(t, cc_force_td(t), '-')
plt.plot(t, pto.force(wec, x_wec, x_opt, nsubsteps), '--')
plt.xlabel('time [s]')
plt.ylabel('force [N]')

plt.figure()
plt.plot(t, cc_vel_td(t)*cc_force_td(t), '-')
plt.plot(t, pto.power(wec, x_wec, x_opt, nsubsteps), '--')
plt.xlabel('time [s]')
plt.ylabel('mechanical power [W]')


In [ ]:
# plot electrical power
plt.figure()
plt.plot(t, cc_current_td(t), '-', label='CC_e')
plt.plot(t, pto.electric_current(
    wec, x_wec, x_opt, nsubsteps), '--', label='PS')
plt.xlabel('time [s]')
plt.ylabel('current [A]')
plt.legend()

plt.figure()
plt.plot(t, cc_voltage_td(t), '-')
plt.plot(t, pto.electric_voltage(wec, x_wec, x_opt, nsubsteps), '--')
plt.xlabel('time [s]')
plt.ylabel('voltage [V]')

plt.figure()
plt.plot(t, cc_current_td(t)*cc_voltage_td(t), '-')
plt.plot(t, pto.electric_power(wec, x_wec, x_opt, nsubsteps), '--')
plt.xlabel('time [s]')
plt.ylabel('electrical power [W]')